<a href="https://colab.research.google.com/github/nsydn/probstat/blob/master/linear_reg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Linear Regression

### The Boston Housing Dataset
The Boston Housing Dataset includes data concerning housing in the towns of Boston, MA. The following describes the dataset columns:

* CRIM: per capita crime rate by town
* RM: average number of rooms per unit
* AGE: proportion of units built before 1940
* DIS: weighted distances to five Boston employment centres
* PTRATIO: pupil-teacher ratio by town
* LSTAT: proportion (\%) of low-educated population
* MEDV: median value of houses (in \$1000 units)

In [ ]:
import numpy as np
import pandas as pd
from google.colab import files
uploaded = files.upload()
import matplotlib.pyplot as plt

In [ ]:
from pandas import read_csv
column_names = ['CRIM', 'RM', 'AGE', 'DIS', 'PTRATIO', 'LSTAT', 'MEDV']
data = read_csv('housing_clean.csv', header=None, names=column_names)
print(data.head(5))

In [ ]:
print(np.shape(data))
print(data.describe())

In [ ]:
# visualize the relationship between the features and the response using scatterplots
fig, axs = plt.subplots(1, 4, figsize=(15, 5))
axs[0].scatter(data.LSTAT, data.MEDV, color='blue'); axs[0].set_xlabel('LSTAT'); axs[0].set_ylabel('MEDV')
axs[1].scatter(data.RM, data.MEDV, color='orange'); axs[1].set_xlabel('RM');
axs[2].scatter(data.DIS, data.MEDV, color='green'); axs[2].set_xlabel('DIS'); 
axs[3].scatter(data.AGE, data.MEDV, color='red'); axs[3].set_xlabel('AGE'); 
# axs[4].scatter(data.CRIM, data.MEDV, color='blue'); axs[4].set_xlabel('CRIM'); 

### Hypothesis
We'd like to propose the following hypothesis:

*   $H_0$: $\beta_1=\beta_2=\dots=\beta_k=0$ (There is no significant relation between $X$ variables and $Y$.)
*   $H_1$: $\beta_j\neq 0$ at least for one $j\in\{1,\dots,k\}$ (There is significant relation between at least one $X$ variable and $Y$.)

### Simple Linear Regression

Simple linear regression is an approach for predicting a quantitative **response** using a **one or more features** (or "regressors" or "predictors" or "input variables" or "independent variables"). It takes the following form:

$y = \beta_0 + \beta_1x_1 + \dots + \beta_nx_n + \epsilon$

- $y$ is the response
- $x_1,\dots,x_n$ are features
- $\beta_0$ is the **population** intercept coefficient
- $\beta_1,\dots,\beta_n$ are the *population* slope coefficients for $x_1,\dots,x_n$
- $\epsilon$ is the **population** error/residual/innovation term

To be able use our model, we must **estimate** the values of these coefficients based on available data (**model training**). And once we've estimated these coefficients, we can use the model to **predict** $y$ values for items that are not in our sample (**model test**).

### Estimating the Model Coefficients (Training the Model)

Now, assume we have a data set of $N$ observations. Model coefficients are generally estimated using the ordinary **Least Squares Method**, which means we are estimating the regression line that minimizes the **sum of squared errors** and therefore maximizes the $R^2$: $$SSE=\sum_{i=1}^N (y_i-\hat{y})^2=\sum_{i=1}^N e_i^2\quad R^2=1-\frac{SSE}{TSS}$$

* **`linregress`**: Python `scipy`'s `linregress` function can only be used for **single-variable** regressions. Alternatively, we can use `statsmodels` to estimate the model coefficients for the house price data:

In [ ]:
# Run a single-variable regression
x=data.RM; y=data.MEDV
from scipy import stats
b1,b0,rho,p,se = stats.linregress(x,y)
print(' Intercept (b0): %.4f\n Slope (b1): %.4f\n P-value: %.4f\n R-squared: %.4f' % (b0,b1,p,rho**2))

In [ ]:
# Plot the regression vs. actual
plt.plot(x, y, 'o', label='actual')
plt.plot(x, b0 + b1*x, 'r', label='fitted')
plt.legend()

* **`statsmodels`**: Getting a detailed regression report is not easy in `stats.linregress`. Instead, we can use `statsmodels` which also allows for **multi-variable** regressions.

In [ ]:
import statsmodels.formula.api as smf

# Define and fit a regression model
model = smf.ols(formula='MEDV ~ RM', data=data)
linreg = model.fit()

# Print the model coefficients or a summary table
print(linreg.params)
print(linreg.summary())

In [ ]:
import statsmodels.formula.api as smf

# Define and fit a regression model
model = smf.ols(formula='MEDV ~ LSTAT + RM + DIS + AGE', data=data)
linreg = model.fit()

# Print the model coefficients or a summary table
print(linreg.params)
print(linreg.summary())

### Interpreting the Model Coefficients

How do we interpret the `LSTAT`, `RM`, `AGE` and `DIS` coefficients ($b_1$, $b_2$, $b_3$ and $b_4$) estimated from our example?
- A unit increase in the proportion of low-status population (`LSTAT`) is **associated with** a `0.4376` unit (in \$1000) decrease in median value of houses (when other variables `RM`, `AGE`, `DIS` are kept constant).
- A unit increase in the average number of rooms per house (`RM`) is **associated with** a `6.1846` unit (in \$1000) increase in median value of houses (when other variables `LSTAT`, `AGE`, `DIS` are kept constant).
- Similar interpretations for $b_3$ and $b_4$ ...


It is good to recall few points:

- $B_0$ and $B_1$ are estimators of population parameters $\beta_0$ and $\beta_1$
- This means $B_0$ and $B_1$ have their own sampling distributions (It is Student's $t$ distribution.).
- Coefficients $b_0$ and $b_1$ are realizations of $B_0$ and $B_1$ estimated from a certain sample
- We can indeed use $b_0$ and $b_1$ to make inferences about $\beta_0$ and $\beta_1$ (e.g., build confidence intervals, carry out hypothesis tests, etc.)
<!-- - But for this class, we will not *manually* calculate $t$ values or derive confidence intervals for $\beta_0$ and $\beta_1$. We'll only interpret outputs from `scipy`, `statsmodels` or `sklearn`. -->

<!-- By default `statsmodels` calculates two-sided 95% confidence intervals for our model coefficients, but we can change it to any $\alpha$ value we want. We can create 90% confidence intervals (which would be be more narrower) or 99% confidence intervals (which would be wider), or whatever intervals we like. -->

* Again, remember that $R^2$ should not be used for comparing models with different number of regressors (because adding new regressors lead to artificially higher values of $R^2$ and causes **overfitting** and this is true even if they are unrelated to the response $Y$). 
<!-- Is that a "good" R-squared value? It's hard to say. The threshold for a good R-squared value depends widely on the domain.  -->
<!-- Therefore, it's most useful as a tool for **comparing different models**. -->

* **R-squared will always increase as you add more features to the model**, even if they are unrelated to the response. Thus, selecting the model with the highest R-squared is not a reliable approach for choosing the best linear model.

* There is alternative to R-squared called **adjusted R-squared** that penalizes model complexity to control for overfitting (Adjusted $R^2$ is reported in the regression summary table obtained using `reg_line.summary()` command.).

In [ ]:
import scipy.stats as stats
#plt.scatter(reg_line.resid,)
qq = stats.probplot(reg_line.resid,dist=stats.norm,plot=plt)

### Using the Model for Prediction

Assume that the median value for a new town with $\{x_1,x_2,x_3,x_4\}=\{20,7,6,50\}$ will be predicted. 

$$\hat{y} = b_0 + b_1x_1 + b_2x_2 + b_3x_3 + b_4x_4$$
<!-- $$\hat{y} = -5.9274 - 0.4376(20) + 6.1846(7) - 0.0493(6) - 0.4698(50)$$ -->

In [ ]:
# First, generate a dataframe for the new observation
x_new = pd.DataFrame({'LSTAT': [20], 'RM': [5], 'DIS': [6], 'AGE': [50]})
print(x_new)

In [ ]:
# Use x_new to predict y
y_hat = linreg.predict(x_new)
print(y_hat)

Thus, we predict a `MEDV` of $\hat{y}=10.96$ (in \$1000 units) for the new town. If we knew that $y=10.50$ for that town, then our prediction error would be $e=0.46$ units for that specific observation. 